In [78]:
from datetime import date

class BookingSystem:
    def __init__(self, booking_id, check_in, check_out, status="Pending"):
        # Initializing booking details with unique identifier and dates
        self.__booking_id = booking_id  # Unique identifier for the booking
        self.__check_in = check_in  # Date when guest checks in
        self.__check_out = check_out  # Date when guest checks out
        self.__status = status  # Current status of booking (Pending, Confirmed, Canceled)
        self.__rooms = []  # Association: Multiple rooms can be in one booking
        self.__invoice = None  # Composition: Each booking has one invoice
        self.__service_requests = []  # Association: Booking can have multiple service requests
        self.__guest = None  # Association: One-to-one relationship with guest

    def set_booking(self):
        #Confirms the booking by changing its status to 'confirmed'.
        # Only pending bookings can be confirmed
        if self.__status.lower() == "pending":
            self.__status = "Confirmed"  # Update status to confirmed
        else:
            raise ValueError("Booking cannot be confirmed from the current status.")

    def cancel_booking(self):
        #Cancel the booking, changing status to 'canceled'.
        # Only pending or confirmed bookings can be canceled
        if self.__status.lower() in ["pending", "confirmed"]:
            self.__status = "Canceled"  # Update status to canceled
            # Make rooms available again when booking is canceled
            for room in self.__rooms:
                room.set_availability(True)
        else:
            raise ValueError("Booking cannot be canceled from the current status.")

    def add_room(self, room):
        # Add a room to this booking if it's available.
        # Check if room is available before adding to booking
        if room.is_available():
            self.__rooms.append(room)  # Add room to booking
            room.set_availability(False)  # Mark room as unavailable
            return True
        return False

    def set_guest(self, guest):
        # Associate a guest with this booking.
        self.__guest = guest  # Set the guest for this booking
        guest.add_booking(self)  # Add this booking to guest's bookings

    def add_service_request(self, service):
        # Add service request to this booking.
        self.__service_requests.append(service)  # Track service requests

    def create_invoice(self, invoice_id, payment_method, amount):
        #Create an invoice for this booking (composition relationship).
        self.__invoice = PaymentAndInvoice(invoice_id, payment_method, amount, False)
        return self.__invoice

    def get_booking(self):
        #Returns the booking details.
        return {
            "booking_id": self.__booking_id,
            "check_in": self.__check_in,
            "check_out": self.__check_out,
            "status": self.__status,
            "rooms": len(self.__rooms)
        }

    def __str__(self):
        #Returns a string representation of the booking.
        return f"Booking ID: {self.__booking_id}, Check-in: {self.__check_in}, Check-out: {self.__check_out}, Status: {self.__status}"



In [79]:
from enum import Enum
class RoomType(Enum):
    #Enum for different types of hotel rooms.
    single = "Single"  # For single occupancy
    double = "Double"  # For double occupancy
    suite = "Suite"    # Luxury suite with additional amenities

class Amenities(Enum):
    #Enum for different amenities available in rooms.
    wifi = "Wi-Fi"      # Internet connectivity
    tv = "TV"           # Television
    minibar = "Mini-Bar"  # Mini refrigerator with beverages

class RoomManagement:
    #This class handles the rooms and their availability.
    def __init__(self, room_num, room_type, amenities, price_per_night, room_availability:bool):
        # Initialize room with its unique details
        self.__room_num = room_num  # Unique room identifier
        self.__room_type = room_type  # Type of room (from RoomType enum)
        self.__amenities = amenities  # List of amenities in the room
        self.__price_per_night = price_per_night  # Cost per night
        self.__room_availability = room_availability  # Whether room is available

    def get_room_info(self):
        # Create string with all room details
        amenities_str = ", ".join([a.value if isinstance(a, Amenities) else str(a) for a in self.__amenities])
        availability = "Available" if self.__room_availability else "Booked"
        return f"Room {self.__room_num}: Type {self.__room_type.value if isinstance(self.__room_type, RoomType) else self.__room_type}, Amenities: {amenities_str}, Price: ${self.__price_per_night}/night, Status: {availability}"

    def is_available(self):
        #Check if room is available.
        return self.__room_availability

    def get_room_num(self):
       #Get room number.
        return self.__room_num

    def set_availability(self, availability):
        #Update room availability status.
        # Used when booking or canceling
        self.__room_availability = availability

    def __str__(self):
      #String representation of room.
        return self.get_room_info()

In [80]:
class GuestManagement:
    #Handles guest information.
    def __init__(self, guest_id, name, email, contact_num, loyalty_status=False, loyalty_points=0):
        # Initialize guest profile with personal details
        self.__guest_id = guest_id  # Unique identifier for guest
        self.__name = name  # Guest's full name
        self.__email = email  # Contact email
        self.__contact_num = contact_num  # Phone number
        self.__loyalty_status = loyalty_status  # Whether enrolled in loyalty program
        self.__loyalty_points = loyalty_points  # Current loyalty points balance
        self.__loyalty_program = None  # Aggregation: Guest can have one loyalty program
        self.__reviews = []  # Aggregation: Guest can have multiple reviews
        self.__bookings = []  # Association: Guest can have multiple bookings

    def get_guest_info(self):
        #Return formatted guest information.
        return f"Guest {self.__guest_id}: {self.__name}, Email: {self.__email}, Contact: {self.__contact_num}, Loyalty: {self.__loyalty_status}, Points: {self.__loyalty_points}"

    def update_loyalty_points(self, points):
        #Add loyalty points to guest's account.
        # Used after stays or special promotions
        self.__loyalty_points += points
        if self.__loyalty_program:
            self.__loyalty_program.add_points(points)

    def enroll_in_loyalty(self, rewards):
        #Enroll guest in loyalty program.
        if not self.__loyalty_status:
            self.__loyalty_status = True
            self.__loyalty_program = LoyaltyProgram(self.__loyalty_points, rewards)
            return True
        return False

    def add_booking(self, booking):
        #Add a booking to guest's history.
        self.__bookings.append(booking)

    def add_review(self, rating, comments):
        #Add a review from this guest.
        review = Reviews(rating, comments)
        self.__reviews.append(review)
        return review

    def get_name(self):
        #Get guest's name.
        return self.__name

    def get_guest_id(self):
        #Get guest's ID.
        return self.__guest_id

    def __str__(self):
        #String representation of guest.
        return self.get_guest_info()

In [81]:
class Rewards(Enum):
    #Enum for different types of loyalty rewards.
    discount = "Discounts"  # Percentage off future stays
    freeNights = "Free Nights"  # Complimentary nights


class LoyaltyProgram:
    #Manages loyalty points and rewards.
    def __init__(self, points, rewards):
        # Initialize loyalty program with starting points and reward options
        self.__points = points  # Current points balance
        self.__rewards = rewards  # Available reward types

    def add_points(self, points):
        # Accumulate points from stays or promotions
        self.__points += points

    def redeem_points(self):
        # Check if enough points are available before redemption
        if self.__points >= 100:
            self.__points -= 100  # Deduct points when redeemed
            return "Reward Redeemed"
        return "Not Enough Points"

    def __str__(self):
        #String representation of loyalty program.
        rewards_str = self.__rewards.value if isinstance(self.__rewards, Rewards) else str(self.__rewards)
        return f"Loyalty Program: {self.__points} Points, Rewards: {rewards_str}"

In [82]:
class Reviews:
    def __init__(self, rating, comments):
        # Store guest feedback about their stay
        self.__rating = rating  # Numerical rating (typically 1-5)
        self.__comments = comments  # Detailed feedback comments

    def get_review(self):
       #Return formatted review.
        # Create string with rating and comments
        return f"Rating: {self.__rating}/5, Comments: {self.__comments}"

    def __str__(self):
       #String representation of review.
        return self.get_review()

In [83]:
class PaymentAndInvoice:
    #Handles payments and invoices.
    def __init__(self, invoice_id, payment_method, amount, payment_status, discount=False, additional_charge=None):
        # Initialize invoice with payment details
        self.__invoice_id = invoice_id  # Unique identifier for invoice
        self.__payment_method = payment_method  # Method used (credit card, cash, etc.)
        self.__amount = amount  # Total amount to be paid
        self.__payment_status = payment_status  # Whether payment is complete
        self.__discount = discount  # Any applicable discounts
        self.__additional_charge = additional_charge  # Extra charges like room service

    def process_payment(self):
        #Process the payment and update status.
        self.__payment_status = True
        return "Payment Processed"

    def get_payment_status(self):
        #Get current payment status.
        return self.__payment_status

    def get_amount(self):
        #Get total invoice amount.
        return self.__amount

    def __str__(self):
        #String representation of invoice.
        status = "Paid" if self.__payment_status else "Pending"
        return f"Invoice {self.__invoice_id}: Amount ${self.__amount}, Payment Status: {status}"



In [84]:
class GuestService:
    #Handles guest service requests.
    def __init__(self, request_type, status=False):
        # Initialize service request details
        self.__request_type = request_type  # Type of service requested (housekeeping, maintenance, etc.)
        self.__status = status  # Whether request is fulfilled

    def update_status(self, status):
        # Mark request as completed or pending
        self.__status = status

    def __str__(self):
        #String representation of service request.
        status = "Completed" if self.__status else "Pending"
        return f"Service Request: {self.__request_type}, Status: {status}"

In [85]:
class Hotel:
    #Main class for managing the hotel system.
    def __init__(self, name):
        # Initialize hotel with name and empty collections
        self.__name = name  # Hotel name
        self.__rooms = {}  # Dictionary of room_num: RoomManagement
        self.__guests = {}  # Dictionary of guest_id: GuestManagement
        self.__bookings = {}  # Dictionary of booking_id: BookingSystem
        self.__next_booking_id = 1000  # Starting ID for bookings
        self.__next_guest_id = 1000  # Starting ID for guests

    def add_room(self, room):
        # Add room to hotel's inventory if not already present
        room_num = room.get_room_num()
        if room_num not in self.__rooms:
            self.__rooms[room_num] = room
            return True
        return False

    def register_guest(self, name, email, contact_num):
        # Registers a new guest
        # Create new guest profile with unique ID
        guest_id = self.__next_guest_id
        self.__next_guest_id += 1
        guest = GuestManagement(guest_id, name, email, contact_num)
        self.__guests[guest_id] = guest
        return guest

    def create_booking(self, guest_id, check_in, check_out, room_nums):
        #This will help us in creating a new booking
        # Verify guest exists
        guest = self.__guests.get(guest_id)
        if not guest:
            raise ValueError(f"Guest with ID {guest_id} not found")

        # Create new booking with unique ID
        booking_id = self.__next_booking_id
        self.__next_booking_id += 1
        booking = BookingSystem(booking_id, check_in, check_out)

        # Associate guest with booking
        booking.set_guest(guest)

        # Add rooms to booking
        for room_num in room_nums:
            room = self.__rooms.get(room_num)
            if not room:
                raise ValueError(f"Room {room_num} not found")
            if not booking.add_room(room):
                raise ValueError(f"Room {room_num} is not available")

        # Store booking
        self.__bookings[booking_id] = booking

        return booking

    def get_room(self, room_num):
        # This will get a room by number
        return self.__rooms.get(room_num)

    def get_booking(self, booking_id):
        # This will get a booking by ID
        return self.__bookings.get(booking_id)

    def __str__(self):
        #Represents the Hotel
        return f"{self.__name} Hotel with {len(self.__rooms)} rooms and {len(self.__guests)} registered guests"


In [86]:
# Example Usage
from datetime import date

def example_usage():
    # Create guest
    guest = GuestManagement(guest_id=12256, name="Nouf Abdulkarim", email="nouf@outlook.com", contact_num="055288900")
    print(guest)

    # Create rooms
    room1 = RoomManagement(room_num=101, room_type=RoomType.single, amenities=[Amenities.wifi, Amenities.tv], price_per_night=755, room_availability=True)
    room2 = RoomManagement(room_num=102, room_type=RoomType.double, amenities=[Amenities.wifi, Amenities.minibar], price_per_night=1000, room_availability=True)
    print(room1)
    print(room2)

    # Create booking
    booking = BookingSystem(booking_id=1, check_in=date(2025, 4, 10), check_out=date(2025, 4, 15))
    booking.set_guest(guest)
    booking.add_room(room1)
    booking.set_booking()
    print(booking)

    # Process payment and generate invoice
    invoice = booking.create_invoice(invoice_id=1, payment_method="Credit Card", amount=500)
    print(invoice)
    invoice.process_payment()
    print(invoice)

example_usage()



Guest 12256: Nouf Abdulkarim, Email: nouf@outlook.com, Contact: 055288900, Loyalty: False, Points: 0
Room 101: Type Single, Amenities: Wi-Fi, TV, Price: $755/night, Status: Available
Room 102: Type Double, Amenities: Wi-Fi, Mini-Bar, Price: $1000/night, Status: Available
Booking ID: 1, Check-in: 2025-04-10, Check-out: 2025-04-15, Status: Confirmed
Invoice 1: Amount $500, Payment Status: Pending
Invoice 1: Amount $500, Payment Status: Paid


In [87]:

# Test Cases
print("\n--- Guest Account Creation ---")
guest1 = Guest(500283, "Sara", "Sara55@gmail.com", "05552887765")
guest2 = Guest(202025, "Nouf", "Noufakb@gmail.com", "0555999876")
assert guest1.get_name() == "Sara"
assert guest2.get_name() == "Nouf"
print(guest1.get_name())
print(guest2.get_name())

print("\n--- Searching for Available Rooms ---")
room1 = Room(101, "Single", ["WiFi"], 500)
room2 = Room(102, "Suite", ["TV", "WiFi"], 700)
assert room1.is_available()
assert room2.is_available()
print(room1)
print(room2)

print("\n--- Making a Room Reservation ---")
booking1 = Booking(1, guest1, r1, date(2025, 4, 1), date(2025, 4, 4))
booking2 = Booking(2, guest2, r2, date(2025, 5, 1), date(2025, 5, 3))
assert booking1.make_reservation()
assert booking2.make_reservation()
print(booking1)
print(booking2)

print("\n--- Invoice Generation ---")
invoice1 = Invoice(1, booking1, "Credit Card", 1500)
invoice2 = Invoice(2, booking2, "Mobile Wallet", 2100)
assert invoice1.amount > 0
assert invoice2.amount > 0
invoice1.generate_invoice()
invoice2.generate_invoice()

print("\n--- Payment Processing ---")
invoice1.process_payment()
invoice2.process_payment()
assert invoice1.get_payment_status()
assert invoice2.get_payment_status()
print(invoice1.get_payment_status())
print(invoice2.get_payment_status())

print("\n--- Reservation History ---")
guest1.view_history()
guest2.view_history()

print("\n--- Cancellation ---")
booking1.cancel_booking()
booking2.cancel_booking()
assert booking1.status == "Cancelled"
assert booking2.status == "Cancelled"
print(booking1)
print(booking2)


--- Guest Account Creation ---
Sara
Nouf

--- Searching for Available Rooms ---
Room 101 (Single) - $500/night
Room 102 (Suite) - $700/night

--- Making a Room Reservation ---
Booking 1 confirmed for Sara
Booking 2 confirmed for Nouf
Booking 1 for Sara in Room 101 (Confirmed)
Booking 2 for Nouf in Room 102 (Confirmed)

--- Invoice Generation ---
Invoice ID: 1
Guest: Sara
Room: 101
Check-in: 2025-04-01
Check-out: 2025-04-04
Total Amount: $1500.00
Status: Unpaid
Invoice ID: 2
Guest: Nouf
Room: 102
Check-in: 2025-05-01
Check-out: 2025-05-03
Total Amount: $2100.00
Status: Unpaid

--- Payment Processing ---
True
True

--- Reservation History ---
Sara's Reservation History:
Booking 1 for Sara in Room 101 (Confirmed)
Nouf's Reservation History:
Booking 2 for Nouf in Room 102 (Confirmed)

--- Cancellation ---
Booking 1 for Sara in Room 101 (Cancelled)
Booking 2 for Nouf in Room 102 (Cancelled)
